In [16]:
import cv2 as cv
import numpy as np

file1=r'C:\Users\luo yong chen\Pictures\box.png'#索引图像
box = cv.imread(file1)
file2 =r'C:\Users\luo yong chen\Pictures\box_in_scene.png'#训练图像
box_in_scene = cv.imread(file2)
cv.imshow("box", box)
cv.imshow("box_in_sence", box_in_scene)

# 创建ORB特征检测器
orb = cv.ORB_create()
kp1, des1 = orb.detectAndCompute(box,None)
kp2, des2 = orb.detectAndCompute(box_in_scene,None)

# 暴力匹配
bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1,des2)

# 绘制匹配
matches = sorted(matches, key = lambda x:x.distance)#根据距离进行排序，找出10个最小的距离，即是最好的匹配
result = cv.drawMatches(box, kp1, box_in_scene, kp2, matches[:10], None)
#cv.imshow("orb-match", result)#绘制前10的匹配项

#根据Lowe的比率测试存储所有符合条件的匹配项
good=[]
for m in matches:
    good.append(m)

#找出透视变换矩阵
src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
M, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC,5.0)
matchesMask = mask.ravel().tolist()
h,w,d = box.shape

#获取box中的四个点
pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
#进行透视变换
dst = cv.perspectiveTransform(pts,M)
box_in_scene = cv.polylines(box_in_scene,[np.int32(dst)],True,(0,255,0),3, cv.LINE_AA)

draw_params = dict(matchColor = (0,255,0), # 用绿色绘制匹配
singlePointColor = None,
matchesMask = matchesMask, # 只绘制内部点
                   flags = 2)
image3=cv.drawMatches(box, kp1, box_in_scene, kp2,good,None,**draw_params)
cv.imshow('result',image3)

cv.waitKey(0)
cv.destroyAllWindows()